<a href="https://colab.research.google.com/github/konfuzio-ai/konfuzio-sdk/blob/master/notebooks/Get_started_with_the_Konfuzio_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get started with the **Konfuzio SDK** 🚀


The [**Konfuzio SDK**](https://dev.konfuzio.com/sdk/index.html#what-is-the-konfuzio-sdk) (Konfuzio Software Development Kit) \
provides a [Python API](https://dev.konfuzio.com/sdk/sourcecode.html#api-reference) to interact with the [Konfuzio Server](https://dev.konfuzio.com/web/index.html#what-is-the-konfuzio-server).

This **notebook shows** how to:
- setup **credentials** you need to connect 🔑
- **install** the SDK in Google Colab 💿
- **initialize** a connection to the Konfuzio Server 🔗
- **run** an example use case 🤓

---



In [ ]:
# @title ## **Credentials** { display-mode: "form" }

# @markdown If you have no account yet, create one [here](https://app.konfuzio.com/accounts/signup/).
# @markdown \
# @markdown \
from getpass import getpass

# @markdown ### Enter user name for Konfuzio Server
User_Name = "" # @param {type:"string"}

# @markdown ### Enter server host url
Host = "https://app.konfuzio.com" # @param {type:"raw"} #dafault: "https://app.konfuzio.com"
Password = getpass('Password you use to login to Konfuzio Server: ')

## **Install** 💿

There are **two** installation **methods**
1. **with** the AI-related dependencies
  ```bash
  pip install konfuzio_sdk
  ```
2. **without** the AI-related dependencies
  ```bash
  pip install konfuzio_sdk[ai]
  ```

By **default**, the SDK is installed **without the AI-related dependencies** like torch or transformers and allows **for using** only the **Data-related SDK concepts** but not the AI models.

**Here** we install the SDK **with AI-related dependencies**, **for using** it **with and without AI components**

In [ ]:
# @title #### **Run intallation** { display-mode: "form" }

# @markdown If you have no account yet, create one [here](https://app.konfuzio.com/accounts/signup/).
# @markdown \
# @markdown \
Method = 'with AI dependencies' # @param ["with AI dependencies", "without AI dependencies"]

print(f"Installing Konfuzio SDK {Method}.")

if Method == 'without AI dependencies':
  # without the AI-related dependencies
  !pip install -q konfuzio_sdk
else:
  # with the AI-related dependencies
  !pip install -q konfuzio_sdk[ai]
print(f"\n[SUCCESS] SDK installed!\n")

## Initialize 🔗

**Initialize a connection** to the Konfuzio Server by passing your credentials **manually** with:

```bash
konfuzio_sdk init
```

or as an alternative **via command line arguments**:
```bash
konfuzio_sdk init --user {User_Name} --password {Password} --host {Host}
```

The init command will create a Token to connect to the Konfuzio Server. This will create variables `KONFUZIO_USER`, `KONFUZIO_TOKEN` and `KONFUZIO_HOST` in an .env file in your working directory.


#### **Run the initialization**

In [ ]:
! konfuzio_sdk init --user {User_Name} --password {Password} --host {Host}


#### **Check installation and initialization**

In [ ]:
try:
  from konfuzio_sdk import KONFUZIO_HOST, KONFUZIO_USER
  print(f"\nSuccesfully initialized the server connection!\nYou are connected with {KONFUZIO_USER} to {KONFUZIO_HOST}.")
except:
  print("\nYou need to restart the Google Colab Session. Go to: Runtime -> Restart Session or press Ctrl+M+. \nThen rerun this cell and continue.\n")

## **Run** 🤓

Run an example use case.

#### **Get all project names and ids**

In [2]:
from konfuzio_sdk.api import get_project_list

In [3]:
# get all projects
projects = get_project_list()
projects = [(p['name'], p['id']) for p in projects]

In [ ]:
# print projects and ids
header = ("Project Name", "Project ID")
print(f"{header[0]:<70} {header[1]:>}")
for name, id in projects:
  print(f"{name:<70} {id:>}")

#### **Get labels and documents form a specific project**

In [5]:
from konfuzio_sdk.data import Project

In [6]:
# @title ##### Enter Project id { display-mode: "form" }
project_id = 14848 # @param { type:"string"} # exchange with the project id of your choice

if not project_id:
  try:
    project_id = projects[0][1]
    print(f"No project id entered, so using the first id which is {project_id}.")
  except:
    print(f"No project could be loaded from Konfuzio Server. Make sure that you have setup a valid Project")

In [7]:
# get project
project = Project(id_=project_id)

In [ ]:
# list project labels
labels = project.labels
for label in labels:
  print(label)

In [ ]:
# list project documents
documents = project.documents
for doc in documents:
  print(doc)